In [146]:
"""
@author: ChewingGumKing_OJF
"""
import json
import os
import sys
import time
from dataclasses import dataclass
from datetime import datetime
#loads necessary libraries
from typing import Union, Any, Tuple, Sequence, List
import re
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import warnings
import requests
from GlobalVariable import *
from GlobalFunctions import GlobalFunctions

error: str = ''
options = webdriver.ChromeOptions()

#options.add_argument("--headless")

path = GlobalVariable.ChromeDriverPath
driver = webdriver.Chrome(options=options, executable_path=path)

def date_transformation(date: str) -> Tuple[str, str]:
    match = re.search(r'(\d{1,2})-(\d{1,2})\s*(\w+)\s*(\d{4})', date)
    if match:
        change = lambda exact: datetime.strptime(exact, '%d %B %Y').strftime('%Y-%m-%d')
        return tuple(map(lambda no: change(' '.join(match.group(no, *(3, 4)))), [1, 2]))

#*******************************************************************************************************************
import logging
def creating_log(script_name: str):

    log_folder_path = 'log_folder'

    if os.path.exists(log_folder_path):
        for files in os.listdir(log_folder_path):
            if files == 'jupyter_run.log':
                os.remove(os.path.join(os.getcwd(), log_folder_path, files))
    else:
        os.makedirs(log_folder_path)

    log_path = os.path.join(os.getcwd(), log_folder_path, f'{script_name}.log')

    logger = logging.getLogger(script_name)
    logger.setLevel(logging.DEBUG)
    log_handler = logging.FileHandler(log_path)
    log_format = logging.Formatter(
        '%(asctime)s -- %(name)s -- %(levelname)s -- %(message)s \n\n')
    log_handler.setFormatter(log_format)
    logger.addHandler(log_handler)
    logger.info('Log reporting is instantiated.')

    return logger


logger = creating_log('jupyter_run')
#*******************************************************************************************************************

/tmp/ipykernel_23536/68414855.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=path)


In [147]:
def split_names(text):
    splitted = text.text.split('\n')
    if len(splitted) == 1:
        splitted.append('')
    return splitted

class ScrapeEvent:
    """ the codebase design uses a Class with it Methods as function scraping singular data(some more,
        in the case of going inside the page just once). It returns the data to a it caller which is handled by a context manager.
    """

    browser: WebDriver = driver
    wait_5sec: WebDriverWait = WebDriverWait(browser, 5)
    error_msg_from_class: str = ''

    def __enter__(self):
        "Handles the contex manager."
        return self

    def __exit__(self, exc_type=None, exc_value=None, exc_tb=None):
        self.browser.quit()

    def get_link(self, link: str) -> None:
        "Hanles the teardown of the context manager."
        self.browser.get(link)
        time.sleep(1)
        self.dispatch('#cookie_action_close_header').click()

    def dispatch(self, locator:str, strategy:webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_element"
        return self.browser.find_element(strategy, locator)

    def dispatchList(self, locator:str, strategy: webdriver = By.CSS_SELECTOR):
        "API calls for Selenium webdriver.find_elements"
        return self.browser.find_elements(strategy, locator)

    def get_done(self) -> Any:
            "Scrapes and return event venue "
            try:
                speaker_list = list(map(split_names, self.dispatchList('.fusion-one-fourth p[style="text-align: center;"]')))
                speaker_list = speaker_list[1:]
                if speaker_list == []:
                    raise IndexError
            except Exception as e:
                try:
                    speaker_list = list(map(split_names, self.dispatchList('.bklyn-team-member-info')))
                except Exception as e:
                    self.error_msg_from_class += '\n' + str(e)
                    logger.error(f'{self.get_events.__name__} Function failed', exc_info=True)
                else:
                    return list(map(lambda a: {'name':a[0], 'title': a[1]}, speaker_list))
            else:
                return list(map(lambda a: {'name':a[0], 'title': a[1]}, speaker_list))



In [148]:
url = 'https://insightevents.se/batterytech/'

In [149]:
with ScrapeEvent() as handler:
    handler.browser.implicitly_wait(5)

    try:
        links = handler.get_link(url)
    except Exception as e:
        logger.error('Connection Error', exc_info=True)
        error += str(e) 
        
    try:
        print('-----********************-----')
        res = handler.get_done()
        print(res)
    except Exception as e:
        logger.error('Function failed', exc_info=True)
        error +=  str(e)

    error += handler.error_msg_from_class


print('-----********************-----')
print('DONE')


-----********************-----
['', '', 'GOTHENBURG']
-----********************-----
DONE


In [150]:
string ='22-23 november 2022'
match = re.search(r'(\d{1,2})-(\d{1,2})\s*(\w+)\s*(\d{4})', string)
if match:
    change = lambda exact: datetime.strptime(exact, '%d %B %Y').strftime('%Y-%m-%d')
    match_dat =map(lambda no: change(' '.join(match.group(no, *(3, 4)))), [1, 2])

print(list(match_dat))


['2022-11-22', '2022-11-23']
